# Student's Dropout Prediction using Supervised Machine Learning Classifiers

In [1]:
# Importing relevant libraries

# Data wrangling
import numpy as np
import pandas as pd
from tqdm import tqdm

# Data Visualization
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# Data pre-processing
from sklearn.preprocessing import StandardScaler

# Data splitting
from sklearn.model_selection import train_test_split

# Machine learning Models
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier


# Evaluation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import roc_curve, auc


#Installing dependencies
!pip install -U kaleido

ModuleNotFoundError: No module named 'tqdm'

In [ ]:
# Read dataset from csv file
data =  pd.read_csv("student's dropout dataset.csv")

# Correcting typographical error for nationality
data.rename(columns = {'Nacionality':'Nationality'}, inplace = True)
data.head()

## Descriptive analysis

In [ ]:
# shape of data
data.shape

We have 4,424 observations (rows) and 35 features (Columns)

In [ ]:
data.info()

The results shows that all columns are numerical except the target column

In [ ]:
# descriptive analysis
pd.set_option('display.max_columns', None)
data.describe(include = 'all')

In [ ]:
#Checking for null values
data.isnull().any().sum()

No null values present in the data set

#Exploratory Data Analysis

The majority of categorical variables in the downloaded dataset have already been converted to numerical format. However, for the purpose of exploratory data analysis (EDA), we will revert certain columns to their original categorical form.

In [ ]:
# Create a copy for exploratory data analysis
data_viz = data.copy()

## Target Feature
First we explore the values in the target column

In [ ]:
data['Target'].unique()

From the target column we can infer the following:
* Dropout: The student dropped out
* Graduate: The student graduated
* Enrolled: The student is currently enrolled

In [ ]:
# Distribution of Target feature
fig = px.pie(values= data_viz['Target'].value_counts(),
             names= data_viz['Target'].value_counts().index.to_list())

fig.update_traces(textposition='inside', textinfo='percent+label',
                  marker=dict(colors=['teal', 'goldenrod', 'slateblue']))

fig.update_layout(showlegend = False, height=400, width=800,
                  title='Distribution of Target')
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

**Observation**
* The graph shows that majority of the students are Graduates **(49%)**
* There are also more Dropouts **(32.1%)** than those currently Enrolled **(17%)**

## Age distribution of students

In [ ]:
# Histogram for Age
fig = px.histogram(data_viz, x='Age at enrollment', color = 'Target',
                   opacity = 0.75, barmode = 'overlay',
                    width = 800, height = 500, color_discrete_sequence=px.colors.qualitative.G10)
fig.update_layout(title='Age distribution of students')
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

**Observation**

* Distribution shows that the data is right skewed indicating that majority of the students are in their late teen's to early 20's
* It is also observed that there was an increase in dropout rate from students in their mid 20's to early 30's

For the next set of data visualizations, we would be dealing with pie charts mostly. To reduce repeatability of code, two functions are created;

One to create dictionaries of all categories with respect to number of students enrolled, students who dropped out or graduated, the other would be to create pie charts based on the dictionaries created for visualization.

In [ ]:
def get_dictionaries(category_list, dfcolumn_name, target_col, dictionary_list):
  '''This function returns a list of dictionaries for value count of each target label per category'''
  for each_category in category_list:
    a_dictionary = dict(data_viz[data_viz[dfcolumn_name]== each_category][target_col].value_counts())
    dictionary_list.append(a_dictionary)
  return dictionary_list

In [ ]:
def make_pie(dictionary_list, trace_list, colors_list, textposition = 'inside'):
  '''This function returns a list of traces for pie charts'''
  for dictionary in dictionary_list:
    trace = go.Pie(values = list(dictionary.values()), labels = list(dictionary.keys()),
           textposition = textposition, textinfo='percent+label',
           marker=dict(colors=colors_list))
    trace_list.append(trace)
  return trace_list



## Gender distribution of students

In [ ]:
# Change gender from numerical to catgorical
data_viz['Gender'] = data_viz['Gender'].map({1:'Male', 0:'Female'})

In [ ]:
# Calculate percentage count for gender
percent_count = round(data_viz['Gender'].value_counts(normalize=True) * 100, 1)

fig = px.bar(percent_count,  orientation= 'h', text_auto=True)

fig.update_traces(marker=dict(color=['red']))

fig.update_layout(height=400, width=700,
                  title='Gender Percentage Count of Students',
                  showlegend = False,
                  font=dict(size=14),
                  xaxis_title = '%count',
                  yaxis_title = 'Gender')
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

In [ ]:
# Getting dictionaries for genders
genders = data_viz['Gender'].unique()
genders_dictionaries = get_dictionaries(genders, 'Gender', 'Target', [])
genders_dictionaries

In [ ]:
# Creating subplots for Gender distribution of students
fig = make_subplots(rows=1, cols=2, subplot_titles=[ 'Male Students', 'Female Students'],
                    specs=[[{'type': 'pie'}, {'type': 'pie'}]])

# Creating empty trace list
traces = []
make_pie(genders_dictionaries, traces, ['teal', 'goldenrod', 'slateblue'])

# Adding pie charts
fig.add_trace(traces[0], row=1, col=1)
fig.add_trace(traces[1], row=1, col=2)

# Updating layout
fig.update_layout(height=500, width=800,
                  title='Gender distribution of students',
                  showlegend = False,
                  font=dict(size=14))

fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

**Observation**
* There was a significant number of female students (64.8%) compared to the males (35.2%).
* Also it is observed that there was a higher rate of dropout students that were male (45.1%), compared to the females (25.1%).

## Students courses

In [ ]:
# Converting the courses back to categorical
data_viz['Course'] = data_viz['Course'].map({1: 'Biofuel Production Technologies',
 2: 'Animation and Multimedia Design', 3: 'Social Service (evening attendance)',
 4: 'Agronomy', 5: 'Communication Design', 6: 'Veterinary Nursing',
 7: 'Informatics Engineering', 8: 'Equiniculture', 9: 'Management',
 10: 'Social Service', 11: 'Tourism', 12: 'Nursing', 13: 'Oral Hygiene',
 14: 'Advertising and Marketing Management', 15: 'Journalism and Communication',
 16: 'Basic Education', 17: 'Management (evening attendance)'})

In [ ]:
# Bar chart for courses
percent_count = data_viz['Course'].value_counts()
fig = px.bar(percent_count,  orientation= 'h', text_auto=True)


fig.update_traces(marker=dict(color=['red','orangered', 'darkorange', 'orange', 'yellow']))
fig.update_layout(height=500, width=1000,
                  title='Courses enrolled by students',
                  showlegend = False,
                  font=dict(size=14),
                  xaxis_title = 'Total',
                  yaxis_title = 'Courses')
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

In [ ]:
# Histogram plot for students courses based on traget status
fig = px.histogram(data_viz, y='Course', color = 'Target',
                   width = 1200, height = 1600,
                   text_auto='.1f', color_discrete_sequence=px.colors.qualitative.Dark2,
                   barnorm = 'percent', barmode = 'group')

# Update layout
fig.update_layout(title='Courses', xaxis_title = '%count')

fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')


**Observation**

Majority of the students are enrolled in Nursing and it also had the lowest droput rate of about **15.4%**.

The course that had the highest dropout rate was **Biofuel Production Technologies (66.7%)** which is also had the least number of enrolled students followed by **Equiniculture (55.3%)**.


## Martial status


In [ ]:
# Converting back to categorical
data_viz['Marital status'] = data_viz['Marital status'].map({1:'Single', 2: 'Married',
                                                             3: 'Widower', 4: 'Divorced',
                                                             5: 'Facto union', 6: 'Legally Separated'})

In [ ]:
# Bar chart for Marital status
count = data_viz['Marital status'].value_counts()
fig = px.bar(count,  orientation= 'h', text_auto=True)

fig.update_traces(marker=dict(color=['red', 'orange']))
fig.update_layout(height=500, width=800,
                  title='Total Marital status of Students',
                  showlegend = False,
                  font=dict(size=14),
                  xaxis_title = 'Total',
                  yaxis_title = 'Marital status')
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

In [ ]:
fig = px.histogram(data_viz, x='Marital status', color = 'Target',
                   width = 1200, height = 500,
                   text_auto='.1f', color_discrete_sequence=px.colors.qualitative.Dark2,
                   barnorm = 'percent', barmode = 'group')

fig.update_layout(title='Marital status', yaxis_title = '%count')

fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

**Observation**

* Vast majority of the students are single, however **30.2% of single students** droppout.
* Another thing to note is that **legally seperated students (66.7)** had the highest percentage of dropouts followed by **Married students (47.2%)**.

For function below is created to simplify the process and reduce repeatability.

In [ ]:
def sub_plots(chart_to_plot, row, col):
  '''This function generates the list of subplots'''
  cols = chart_to_plot * col
  rows = [cols] * row
  return rows


## Student's Financial Status

Next, we will analyze the financial status of students to understand how scholarship status, debt status, and tuition payment status correlate with dropout rates.



In [ ]:
# Converting columns back to categorical
to_convert = ['Debtor', 'Tuition fees up to date', 'Scholarship holder']
for i in to_convert:
    data_viz[i] = data_viz[i].map({1:'Yes', 0: 'No'})

In [ ]:
# Getting dictionaries for students in debt/ not in debt
debt = data_viz['Debtor'].unique()
debt_dictionaries = get_dictionaries(debt, 'Debtor', 'Target', [])
debt_dictionaries

In [ ]:
# Getting dictionaries for students with tuition fees paid/ not paid
tuition = data_viz['Tuition fees up to date'].unique()
tuition_dictionaries = get_dictionaries(tuition, 'Tuition fees up to date', 'Target', [])
tuition_dictionaries

In [ ]:
# Getting dictionaries for students with scholarship/ no scholarship
scholarship = data_viz['Scholarship holder'].unique()
scholarship_dictionaries = get_dictionaries(scholarship, 'Scholarship holder', 'Target', [])
scholarship_dictionaries

In [ ]:
# Creatign subplots for pie charts
fig = make_subplots(rows=3, cols=2,
                    subplot_titles = ['No Debt', 'Debt', 'Tuition Paid', 'Tuition not paid', 'No Scholarship', 'Scholarship'],
                    specs= sub_plots([{'type':'pie'}], 3,2), vertical_spacing = 0.07)

# Initialize an empty list for traces
traces = []

# Generate traces
make_pie(debt_dictionaries, traces, ['teal', 'goldenrod', 'slateblue'])
make_pie(tuition_dictionaries, traces, ['teal', 'goldenrod', 'slateblue'])
make_pie(scholarship_dictionaries, traces, ['teal', 'goldenrod', 'slateblue'])

# Adding pie charts to fig
fig.add_trace(traces[0], row=1, col=1)
fig.add_trace(traces[1], row=1, col=2)
fig.add_trace(traces[2], row=2, col=1)
fig.add_trace(traces[3], row=2, col=2)
fig.add_trace(traces[4], row=3, col=1)
fig.add_trace(traces[5], row=3, col=2)


fig.update_layout(title="Students Financial status", height=1000, width=1000, showlegend = False, title_pad=dict(t=100))
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

**Observation**

Unsurprisingly, students who were in **debt** and had not completed payment for **tuition** had a higher dropout rate of **62% and 86.6%** respectively.

Similarly, students who were granted **scholarships** had a low dropout rate of **12.2%** compared to those who were not given **(38.7%)**.

Based on our EDA, we have a good understanding of how the data is distributed by gender and age, as well as how certain features like courses enrolled and financial status correlate with the dropout rate. In the next phase, we will build and train a classification model to see if our findings are consistent with the model's results.

## Data preparation

### Features selection

The easiest way to view how independent features are correlated is through correlation analysis.

This will help in feature selection and determine which independent variables can be dropped or combined to avoid Multicollinearity

We can visualize this using heatmaps

In [ ]:
# Heat map showing correlation of features
fig = px.imshow(data.drop('Target', axis = 1).corr().round(2), text_auto=True, aspect="auto", color_continuous_scale='RdBu_r')
fig.update_layout(title = 'Correlation Analysis for independent features', height=800, width=1500)
fig.write_image('fig.png', engine='kaleido')
fig.show('png')

**Observation**:

From the results we can see some features have strong correlation with each other:
* Nationality and International
* Mother's qualification and Father's qualification
* Mother's occupation and Father's occupation
* Curricular Units 1st sem and Curricular Units 2nd sem

We can decide to pick or drop either `Nationality or International`, `Mother's qualification or Father's qualification` and `Mother's occupation or Father's occupation`.

As for `Curricular Units 1st sem` and `Curricular Units 2nd sem` we need to retain this information in our data set without losing valuable information or risk the model overfitting.

To solve this, Principal Component Analysis is done to reduce dimensionality.


In [ ]:
# Extract columns for PCA
data_forPCA = data[['Curricular units 1st sem (credited)',
          'Curricular units 1st sem (enrolled)',
          'Curricular units 1st sem (evaluations)',
          'Curricular units 1st sem (without evaluations)',
          'Curricular units 1st sem (approved)',
          'Curricular units 1st sem (grade)',
          'Curricular units 2nd sem (credited)',
          'Curricular units 2nd sem (enrolled)',
          'Curricular units 2nd sem (evaluations)',
          'Curricular units 2nd sem (without evaluations)',
          'Curricular units 2nd sem (approved)',
          'Curricular units 2nd sem (grade)']]


# PCA with one component
pca = PCA(n_components=1)

# Fit PCA to data and transform it
pca_result = pca.fit_transform(data_forPCA)

# Create a new DataFrame with the reduced feature
df_pca = pd.DataFrame(data=pca_result, columns=['PCA Feature']).squeeze()
data['Curricular 1st and 2nd sem PCA'] = df_pca

Next, we drop the following features due to high correlation:
* Nationality
* Mother's occupation
* Father's Qualification
* Curricular units 1st sem (enrolled)
* Curricular units 1st sem (credited)
* Curricular units 1st sem (evaluations)
* Curricular units 1st sem (approved)
* Curricular units 1st sem (grade)
* Curricular units 1st sem (without evaluations)
* Curricular units 2nd sem (credited)
* Curricular units 2nd sem (enrolled)
* Curricular units 2nd sem (evaluations)
* Curricular units 2nd sem (approved)
* Curricular units 2nd sem (without evaluations)
* Curricular units 2nd sem (grade)


In [ ]:
# Dropping features
data.drop(['Nationality', 'Mother\'s occupation', 'Father\'s qualification',
          'Curricular units 1st sem (credited)',
          'Curricular units 1st sem (enrolled)',
          'Curricular units 1st sem (evaluations)',
          'Curricular units 1st sem (without evaluations)',
          'Curricular units 1st sem (approved)',
          'Curricular units 1st sem (grade)',
          'Curricular units 2nd sem (credited)',
          'Curricular units 2nd sem (enrolled)',
          'Curricular units 2nd sem (evaluations)',
          'Curricular units 2nd sem (without evaluations)',
          'Curricular units 2nd sem (approved)',
          'Curricular units 2nd sem (grade)', 'Inflation rate', 'GDP',
           'Unemployment rate'], axis = 1, inplace = True)

In [ ]:
# Heat map showing correlation of features
fig = px.imshow(data.drop('Target', axis = 1).corr().round(2), text_auto=True, aspect="auto", color_continuous_scale='RdBu_r')
fig.update_layout(title = 'Correlation Analysis for independent features', height=800, width=1500)
fig.write_image('fig.png', engine='kaleido')
fig.show('png')

### Data Processing
 Before standardizing the data we need transform the target feature from categorical to numerical data.

In [ ]:
# Get dummies for Target columns
dummies = pd.get_dummies(data['Target'])

#Drop all columns except that for Dropout
dummies.drop(['Enrolled',	'Graduate'], axis = 1, inplace= True)
data['Target'] = dummies
data.head()

In [ ]:
# Assigning x and y features
y = np.array(data['Target'])
X_features = data.drop('Target', axis = 1)
X_features.head()

In [ ]:
y[:5]

In [ ]:
# Normalizing data
scaler =  StandardScaler()
X = scaler.fit_transform(X_features)
X


## Model Building

Because the target variable is categorical (either Dropout or not), this is a classification problem.
We will train five supervised machine learning classification models:

* Logistic regression
* Decision trees
* Support vector machines
* Random forest
* K-nearest neighbors

First, we will split our data into training, validation, and test sets. We will set the random state to 42 to ensure reproducibility.

### Splitting Training and Test set

In [ ]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Creating validation set

In [ ]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

### Baseline Model Accuracy

In [ ]:
# Calculate the baseline accuracy to beat
target_value, frequency = np.unique(y, return_counts = True)
print("Unique Values:", target_value)

print("Frequency Values:", frequency)

# Maximum frequency value
max_val = frequency.max()
length_y = len(y)

print('\n')

baseline_acc = round((max_val/length_y) * 100, 3)
print(f'The baseline accuracy to beat is {baseline_acc}%')



A list is created to store all the trained models with the best hyperparameters

In [ ]:
# List of all trained models

trained_models = []

### Logistic Regression

In [ ]:
# Train a logistic regression model
lr_model = LogisticRegression(random_state = 42, max_iter= 1000)
lr_model.fit(X_train, y_train)


# Predict target values for train data
y_pred_train = lr_model.predict(X_train)

# Predict target values for validation data
y_pred_val = lr_model.predict(X_val)

# Predict target values for test data
y_pred = lr_model.predict(X_test)

# Evaluate the model's accuracy
lr_acc_train = round(accuracy_score(y_train, y_pred_train), 3)
lr_acc_val = round(accuracy_score(y_val, y_pred_val), 3)
lr_acc = round(accuracy_score(y_test, y_pred), 2)
print(f'Accuracy of logistic regression model on the training set is {lr_acc_train * 100}%')
print(f'Accuracy of logistic regression model on the validation set is {lr_acc_val * 100}%')
print('\n')
print(f'Accuracy of logistic regression model on the test set is {lr_acc * 100}%')

# Add to model list
trained_models.append(lr_model)


### Decision Trees

In [ ]:
# List for possible depth value
depth_values = list(range(1, 21))

# Create empty list for training data and validation data
training_acc = []
validation_acc = []

for d in tqdm(depth_values):
  tree_model = DecisionTreeClassifier(max_depth=d, random_state=42)
  tree_model.fit(X_train, y_train)

  # Training accuracy
  y_pred_training = tree_model.predict(X_train)
  training_acc.append(accuracy_score(y_train, y_pred_training))

  # Validation accuracy
  y_pred_validation = tree_model.predict(X_val)
  validation_acc.append(accuracy_score(y_val, y_pred_validation))


In [ ]:
# Plotting learning curve
plt.plot(training_acc, label = 'Training Accuracy')
plt.plot(validation_acc, label = 'Validation Accuracy')
plt.xlabel('Depth Values')
plt.ylabel('Accuracy Score')
plt.title('Learning curve')
plt.legend();

From the training curve we can estimate that the best depth value that yielded the highest accuracy score was between 2 and 5. However, a better way to pick out the depth value would be to get the index value that corresponds to the highest accuracy score in the `validation_acc` list

In [ ]:
# Best depth value
best_val_acc = max(validation_acc)
best_depth = validation_acc.index(best_val_acc) +1
print(f'Best depth value with an accuracy of {round(best_val_acc * 100, 2)}% on the validation set is {best_depth}')


In [ ]:
# Train a Decision tree model with best depth
tree_model = DecisionTreeClassifier(max_depth=best_depth, random_state=42)
tree_model.fit(X_train, y_train)

# Predict target values for test data
y_pred = tree_model.predict(X_test)

# Evaluate the model's accuracy
tree_acc = round(accuracy_score(y_test, y_pred), 3)
print(f'Accuracy of Decision tree model is {tree_acc * 100}%')


# Add to model list
trained_models.append(tree_model)

### Support Vector Machines

In [ ]:
# Iterating to see which kernel gives the best result
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
score_list = {}
scores = 0
svm_linear = []

for k in tqdm(kernels):
    svm_model = SVC(random_state = 42, kernel= k)
    svm_model.fit(X_train, y_train)
    if k == 'linear':
      svm_linear.append(svm_model)
    f_score = svm_model.score(X_val, y_val)
    score_list.update({k: f_score})


score_list

In [ ]:
max_val = max(score_list.values())
list_values = list(score_list.values())
list_keys = list(score_list.keys())

# Best kernel
k = list_keys[list_values.index(max_val)]

In [ ]:
from IPython.display import Markdown
Markdown(f"""
#### From the result we can determine that the optimal kernal for the SVC is {k}""")

In [ ]:
# Train a SVC model
svm_model = SVC(random_state = 42, kernel = k)
svm_model.fit(X_train, y_train)

# Predict target values for test data
y_pred = svm_model.predict(X_test)

# Evaluate the model's accuracy
svm_acc = round(accuracy_score(y_test, y_pred),3)
print(f'Accuracy of Support vector classifier model is {svm_acc * 100}%')

# Add to model list
trained_models.append(svm_model)

To train both Random forest and K-nearest neighbours, we will make use of Grid Search to determine the best parameters. We will also stick to the default value for cross validation which is 5 folds.

### Random forest

In [ ]:
# Train a Random forest model
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# Predict target values for training data
y_pred_train = rf_model.predict(X_train)
rf_acc_train = round(accuracy_score(y_train, y_pred_train), 3)

# Predict target values for validation data
y_pred_val = rf_model.predict(X_val)
rf_acc_val = round(accuracy_score(y_val, y_pred_val), 3)

print('With default parameters: ')
print(f'Accuracy of Random forest model on training data is {rf_acc_train * 100}%')
print(f'Accuracy of Random forest model on validation data is {rf_acc_val * 100}%')


We can perform a grid search to generate best parameters to train a random forest model.

In [ ]:
# Using Grid search to determine best parameters
param_grid = {
    'n_estimators': [25, 50, 75, 100],
    'max_features': ['sqrt', 'log2', None],
    'max_depth': [3, 6, 9],
    'max_leaf_nodes': [3, 6, 9],
}

grid_search = GridSearchCV(RandomForestClassifier(random_state=42),
                           param_grid=param_grid, cv = 5)
# Fit Grid
grid_search.fit(X_train, y_train)
best_rf_parameters = grid_search.best_params_
print(f'Grid search parameters: {best_rf_parameters}')



In [ ]:
# Train a Random forest model with best estimator from Grid search
grid_rf_model = RandomForestClassifier(random_state = 42,
                                       max_depth = best_rf_parameters['max_depth'],
                                      max_leaf_nodes = best_rf_parameters['max_leaf_nodes'],
                                      n_estimators = best_rf_parameters['n_estimators'],
                                       max_features = best_rf_parameters['max_features'])
grid_rf_model.fit(X_train, y_train)

# Predict target values for training data
y_pred_train = grid_rf_model.predict(X_train)
rf_acc_train = round(accuracy_score(y_train, y_pred_train), 3)

# Predict target values for validation data
y_pred_val = grid_rf_model.predict(X_val)
rf_acc_val = round(accuracy_score(y_val, y_pred_val), 3)

print('With Grid search best estimator parameters: ')
print(f'Accuracy of Random forest model on training data is {rf_acc_train * 100}%')
print(f'Accuracy of Random forest model on validation data is {rf_acc_val * 100}%')

In [ ]:
# Predict target values for test data using default parameters
y_pred = grid_rf_model.predict(X_test)
rf_acc = round(accuracy_score(y_test, y_pred), 3)
print(f'Accuracy of Random forest model on test data is {rf_acc * 100}%')

# Add to model list
trained_models.append(grid_rf_model)

### K- Nearest Neighbours



In [ ]:
# Define the best parameter values using grid search
param_grid = {'n_neighbors': np.arange(1, 25)}

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)

# Fit grid
grid_knn.fit(X_train, y_train)
best_knn_parameters = grid_knn.best_params_
print(f'Grid search parameters: {best_knn_parameters}')


In [ ]:
# Train Knn model with grid parameters
grid_knn_model = KNeighborsClassifier(n_neighbors= best_knn_parameters['n_neighbors'])
grid_knn_model.fit(X_train, y_train)
# Predict target values for training data
y_pred_train = grid_knn_model.predict(X_train)
knn_acc_train = round(accuracy_score(y_train, y_pred_train), 3)

# Predict target values for validation data
y_pred_val = grid_knn_model.predict(X_val)
knn_acc_val = round(accuracy_score(y_val, y_pred_val), 3)

print('With Grid search best estimator parameters: ')
print(f'Accuracy of KNN model on training data is {knn_acc_train * 100}%')
print(f'Accuracy of KNN model on validation data is {knn_acc_val * 100}%')

We will also try the elbow method to compare optimal K value

In [ ]:
# Empty list for accuracy scores
train_accuracy = []
val_accuracy = []
for i in tqdm(range(1,21)):
  # Train a Knn model
  knn_model = KNeighborsClassifier(n_neighbors=i)
  # Fit model
  knn_model.fit(X_train, y_train)

  # Prediction for training data
  y_pred_train = knn_model.predict(X_train)
  # Prediction for validation data
  y_pred_val = knn_model.predict(X_val)

  # Accuracy Scores
  train_accuracy.append(accuracy_score(y_train, y_pred_train))
  val_accuracy.append(accuracy_score(y_val, y_pred_val))

In [ ]:
# Plotting learning curve
plt.plot(train_accuracy, label = 'Training Accuracy')
plt.plot(val_accuracy, label = 'Validation Accuracy')
plt.xlabel('n Values')
plt.ylabel('Accuracy Score')
plt.title('Learning curve')
plt.legend();

From the curve, we can estimate that the best k values are between 12 and 17.

In [ ]:
# Determine best n value
n = val_accuracy.index(max(val_accuracy)) + 1

In [ ]:
print(f'From the result we can determine that the optimal n value with the highest score of {round(max(val_accuracy), 3)  * 100}% is {n}')

We will use the best parameters retrieved from grid search.

In [ ]:
# Train a KNN model
knn_model = KNeighborsClassifier(n_neighbors=n)
# knn_model = KNeighborsClassifier(n_neighbors=21)
knn_model.fit(X_train, y_train)

# Predict target values for test data
y_pred = knn_model.predict(X_test)

# Evaluate the model's accuracy
knn_acc = round(accuracy_score(y_test, y_pred), 3)
print(f'Accuracy of KNN model is {knn_acc * 100}%')

# Confusion Matrix
knn_matrix = confusion_matrix(y_test, y_pred)

# Add to model list
trained_models.append(knn_model)

## Accuracy for each classification model

For each model we would view which of them performs the best.

In [ ]:
# Bar plot showing each classifier
accuracy_of_models = {'SVC': svm_acc,
                      'Random Forest': rf_acc,
                      'Logistic Regression': lr_acc,
                      'KNN': knn_acc,
                      'Decision Tree': tree_acc}


fig = px.bar(y = list(accuracy_of_models.keys()), x= list(accuracy_of_models.values()),
             color = list(accuracy_of_models.values()),
             width = 800, height = 400,
             color_discrete_sequence=px.colors.qualitative.G10,
             labels={'y':'Classifier', 'x':'Accuracy'}, text_auto=True)


fig.update_layout(title='Accuracy performance of classification models',
                  showlegend=False)
fig.write_image('fig.svg', engine='kaleido')
fig.show('svg')

## Confusion maxtix of the best classifier

In [ ]:
list_of_models = ['Logistic Regression','Decision Tree', 'SVC','Random Forest', 'KNN']
dictionary_models = {list_of_models[i]: trained_models[i] for i in range(len(list_of_models))}
dictionary_models

In [ ]:
# Find the highest score
scores = [svm_acc, rf_acc, lr_acc, knn_acc, tree_acc]
best_score = max(scores)
key_best_model = ''
best_model = ''

# Generate a heatmap for the classifier that has the highest score
for key, value in accuracy_of_models.items():
  if best_score == value:
    key_best_model = key
    best_model = dictionary_models[key_best_model]
    ConfusionMatrixDisplay.from_estimator(best_model,
                                          X_test, y_test,
                                          display_labels=['Not dropout', 'Dropout'])
    plt.title(key_best_model);

    #Pop the best performing model from dictionary
    dictionary_models.pop(key_best_model)




### Confusion matrix for other classifiers

In [ ]:
colors = ['BuPu', 'CMRmap', 'gnuplot', 'cividis']
position = 0

fig, ax = plt.subplots(1, 4, figsize=(20, 20))

for key, value in dictionary_models.items():
    disp = ConfusionMatrixDisplay.from_estimator(value,
                                                 X_test, y_test,
                                                 display_labels=['Not dropout', 'Dropout'],
                                                 cmap=colors[position], ax=ax[position], colorbar=False)

    # Add title
    ax[position].set_title(key)

    position += 1

plt.tight_layout()  # Adjust spacing between subplots
plt.show()



In [ ]:
Markdown(f"""
#### From the results above we can see that {best_model} perfoms best with the highest accuracy of {round(best_score * 100, 2)}%""")


## Results

### Feature Importances

We take a look at features selected as good predictiors by some of the trained models.


In [ ]:
# create list for features
feature = X_features.columns

In [ ]:
# Logistic regression
importances = lr_model.coef_[0]
lr_feature_importance = pd.Series(np.exp(importances), index = feature).sort_values()

# Decision tree
dt_importance = pd.Series(np.exp(tree_model.feature_importances_), index = feature).sort_values()

# Random forest
rf_importance = pd.Series(np.exp(rf_model.feature_importances_), index = feature).sort_values()

# SVM linear kernel
svm_importance = pd.Series(np.exp(svm_linear[0].coef_[0]), index = feature).sort_values()



In [ ]:
# Create subplots
fig, ax = plt.subplots(2, 2, figsize=(15, 8))

# Add the barplots
lr_feature_importance.plot(kind = 'barh', ax = ax[0][0], title = 'LR importance')
dt_importance.plot(kind = 'barh', ax = ax[0][1], title = 'Decision Trees importance')
rf_importance.plot(kind = 'barh', ax = ax[1][0], title = 'Random forest importance')
svm_importance.plot(kind='barh', ax = ax[1][1], title = 'SVM importance')

# Add vertical lines at x=1
for axes_row in ax:
    for axes in axes_row:
        axes.axvline(x=1, color='black')
plt.tight_layout()
plt.show()


When using logistic regression, SVM, random forest, and decision trees to predict outcomes, it was found that the features "Courses" and "Age at enrollment" were very important, with values greater than 1. This means that these features have a significant impact on predicting outcomes. It is important to consider these factors carefully when designing models for accurate predictions in similar contexts.

In other words, the two features "Courses" and "Age at enrollment" are strong predictors of the outcome, and they should be included in any model that aims to accurately predict outcomes.

In [ ]:
# ROC curve for best model's predictions
y_pred = best_model.predict(X_test)
fpr, tpr, threshold = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)

# Plot ROC curve
plt.figure()
plt.plot(fpr, tpr)
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.show()
print('\n')
# Print the AUC score
print(f'The AUC score is :{round(roc_auc, 2)}')

An AUC score of 0.78 means that the model is able to correctly classify 78% of the positive cases and 22% of the negative cases. This is considered to be good performance, but not excellent. There is still room for improvement

In [ ]:
import joblib
joblib.dump(grid_rf_model, 'grid_rf_model.pkl')  # Save the trained model
